### Importing  Libraries

#### Steps To Be Followed
1. Importing necessary Libraries
2. Creating S3 bucket 
3. Mapping train And Test Data in S3
4. Mapping The path of the models in S3

In [78]:
import sagemaker
import boto3
from sagemaker.amazon.amazon_estimator import get_image_uri 
from sagemaker.session import s3_input, Session
from sagemaker import image_uris

In [79]:
bucket_name = 'banking-app-demo' 
my_region = boto3.session.Session().region_name 
print(my_region)

ap-south-1


In [75]:
s3 = boto3.resource('s3')
try:
    if  my_region == 'ap-south-1':
        s3.create_bucket(Bucket=bucket_name)
    print('S3 bucket created successfully')
except Exception as e:
    print('S3 error: ',e)
    
    
# If it doesn't work create a bucket manually in s3 bucket    

S3 error:  An error occurred (IllegalLocationConstraintException) when calling the CreateBucket operation: The unspecified location constraint is incompatible for the region specific endpoint this request was sent to.


In [80]:
# set an output path where the trained model will be saved
prefix = 'xgboost-as-a-built-in-algo'
output_path ='s3://{}/{}/output'.format(bucket_name, prefix)
print(output_path)

s3://banking-app-demo/xgboost-as-a-built-in-algo/output


#### Downloading The Dataset And Storing in S3

In [81]:
import pandas as pd
import urllib
try:
    urllib.request.urlretrieve ("https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv", "bank_clean.csv")
    print('Success: downloaded bank_clean.csv.')
except Exception as e:
    print('Data load error: ',e)

try:
    model_data = pd.read_csv('./bank_clean.csv',index_col=0)
    print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)

Success: downloaded bank_clean.csv.
Success: Data loaded into dataframe.


In [82]:
### Train Test split

import numpy as np
train_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data))])
print(train_data.shape, test_data.shape)

(28831, 61) (12357, 61)


In [26]:
!pip install --upgrade sagemaker

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 854.4/854.4 kB 53.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached attrs-23.1.0-py3-none-any.whl (61 kB)
  Using cached PyYAML-6.0-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (682 kB)
  Created wheel for sagemaker: filename=sagemaker-2.173.0-py2.py3-none-any.whl size=1163282 sha256=f1a5a87086f8e6698a5adf13a6be35e617d2ada8c60b3e236e5228f3a5bed1da
  Stored in directory: /home/ec2-user/.cache/pip/wheels/cc/23/21/6857de3905209ac2dbbe974e18a5654d28cd247f4223ff6bff
Successfully built sagemaker
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 5.4.1
    Uninstalling PyYAML-5.4.1:
      Successfully uninstalled PyYAML-5.4.1
  Attempting uninstall: attrs
    Found existing installation: attrs 22.2.0
    Uninstalling attrs-22.2.0:
      Successfully uninstalled attrs-22.

In [83]:
### Saving Train And Test Into Buckets
## We start with Train Data
import os
pd.concat([train_data['y_yes'], train_data.drop(['y_no', 'y_yes'], 
                                                axis=1)], 
                                                axis=1).to_csv('train.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
s3_input_train = sagemaker.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), content_type='csv')

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


In [84]:
# Test Data Into Buckets
pd.concat([test_data['y_yes'], test_data.drop(['y_no', 'y_yes'], axis=1)], axis=1).to_csv('test.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'test/test.csv')).upload_file('test.csv')
s3_input_test = sagemaker.TrainingInput(s3_data='s3://{}/{}/test'.format(bucket_name, prefix), content_type='csv')

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


### Building Models Xgboot- Inbuilt Algorithm

In [85]:
# this line automatically looks for the XGBoost image URI and builds an XGBoost container.
# specify the repo_version depending on your preference.
# container = ImageURIProvider(boto3.Session().region_name,
#                           'xgboost', 
#                           repo_version='1.0-1')

container = sagemaker.image_uris.retrieve("xgboost", boto3.Session().region_name, "1.7-1")


INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


In [86]:
# initialize hyperparameters
hyperparameters = {
        "max_depth":"5",
        "eta":"0.2",
        "gamma":"4",
        "min_child_weight":"6",
        "subsample":"0.7",
        "objective":"binary:logistic",
        "num_round":50
        }

In [87]:
# construct a SageMaker estimator that calls the xgboost-container
estimator = sagemaker.estimator.Estimator(image_uri=container, 
                                          hyperparameters=hyperparameters,
                                          role=sagemaker.get_execution_role(),
                                          train_instance_count=1, 
                                          train_instance_type='ml.m5.2xlarge', 
                                          train_volume_size=5, # 5 GB 
                                          output_path=output_path,
                                          train_use_spot_instances=True,
                                          train_max_run=300,
                                          train_max_wait=600)



See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [88]:
estimator.fit({'train': s3_input_train,'validation': s3_input_test})


INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2023-07-17-14-55-27-085


2023-07-17 14:55:27 Starting - Starting the training job...
2023-07-17 14:55:42 Starting - Preparing the instances for training.........
2023-07-17 14:57:24 Downloading - Downloading input data
2023-07-17 14:57:24 Training - Downloading the training image...
2023-07-17 14:57:55 Training - Training image download completed. Training in progress....
2023-07-17 14:58:25 Uploading - Uploading generated training model[2023-07-17 14:58:19.129 ip-10-0-72-225.ap-south-1.compute.internal:7 INFO utils.py:28] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2023-07-17 14:58:19.152 ip-10-0-72-225.ap-south-1.compute.internal:7 INFO profiler_config_parser.py:111] User has disabled profiler.
[2023-07-17:14:58:19:INFO] Imported framework sagemaker_xgboost_container.training
[2023-07-17:14:58:19:INFO] Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
[2023-07-17:14:58:19:INFO] No GPUs detected (normal if no gpus installed)
[2023-07-17:14:58:19:INFO] Running XGBoost 

### Deploy Machine Learning Model As Endpoints

In [89]:
xgb_predictor = estimator.deploy(initial_instance_count=1,instance_type='ml.m4.xlarge')

INFO:sagemaker:Creating model with name: sagemaker-xgboost-2023-07-17-14-59-10-244
INFO:sagemaker:Creating endpoint-config with name sagemaker-xgboost-2023-07-17-14-59-10-244
INFO:sagemaker:Creating endpoint with name sagemaker-xgboost-2023-07-17-14-59-10-244


-----!

#### Prediction of the Test Data

In [90]:
# from sagemaker.predictor import csv_serializer
# from sagemaker.serializers import csv_serializer
from sagemaker.serializers import CSVSerializer


test_data_array = test_data.drop(['y_no', 'y_yes'], axis=1).values #load the data into an array
xgb_predictor.content_type = 'text/csv' # set the data type for an inference
csv_serializer = CSVSerializer()

xgb_predictor.serializer = csv_serializer # set the serializer type
predictions = xgb_predictor.predict(test_data_array).decode('utf-8') # predict!


predictions_array = np.fromstring(predictions[1:], sep=',') # and turn the prediction into an array

print(predictions_array.shape)

(1,)


/tmp/ipykernel_9017/2374047114.py:14: DeprecationWarning: string or file could not be read to its end due to unmatched data; this will raise a ValueError in the future.
  predictions_array = np.fromstring(predictions[1:], sep=',') # and turn the prediction into an array


In [91]:
predictions_array

array([0.05214286])

In [ ]:
cm = pd.crosstab(index=test_data['y_yes'], columns=np.round(predictions_array), rownames=['Observed'], colnames=['Predicted'])
tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Purchase", "Purchase"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("No Purchase", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Purchase", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))

#### Deleting The Endpoints

In [97]:
sagemaker.Session().delete_endpoint(xgb_predictor.endpoint)
bucket_to_delete = boto3.resource('s3').Bucket(bucket_name)
bucket_to_delete.objects.all().delete()

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
INFO:sagemaker:Deleting endpoint with name: sagemaker-xgboost-2023-07-17-14-59-10-244


[{'ResponseMetadata': {'RequestId': '1Z4HY5XG9VG1NSCW',
   'HostId': 'f9ZtZ46chzklmU18h3xAjEu7lSbsQ0U1YGU+lGh3DXYjLQ2pnQqNyTGrNshYM3lYN/E3a+1oEMM=',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': 'f9ZtZ46chzklmU18h3xAjEu7lSbsQ0U1YGU+lGh3DXYjLQ2pnQqNyTGrNshYM3lYN/E3a+1oEMM=',
    'x-amz-request-id': '1Z4HY5XG9VG1NSCW',
    'date': 'Mon, 17 Jul 2023 15:07:36 GMT',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3',
    'connection': 'close'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2023-07-17-14-55-27-085/debug-output/index/000000000/000000000000_worker_0.json'},
   {'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2023-07-17-14-55-27-085/debug-output/training_job_end.ts'},
   {'Key': 'xgboost-as-a-built-in-algo/train/train.csv'},
   {'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2023-07-17-14-55-27-085/profiler-output/system/incremental/2023071